# Skull Stripping Segmentation NFBS Data

My plan is to use the Neurofeedback Skull-stripped (NFBS) dataset to train a 3D UNet model on that. Then use that trained 3D UNet model to do automated skull stripping on the ATLAS 2.0 dataset later in our lesion segmentation notebook and source code. Skull stripping is a preprocessing step before lesion segmentation.

Reference: https://www.analyticsvidhya.com/blog/2021/06/introduction-to-skull-stripping-image-segmentation-on-3d-mri-images/

MRI Brain Dataset with skull stripped labels: http://preprocessed-connectomes-project.org/NFB_skullstripped/

## NFBS Dataset

- 125 participants 21 to 45 years old with a variety of clinical and sub-clinical psychiatric symptoms.
- Structural T1-weighted anonymized (de-faced) images) with a single channel.
- Brain mask is the ground truth obtained using Beast (Brain extraction based on nonlocal segmentation) method and applying manual edits by domain experts to remove non-brain tissue.
- Skull-stripped image is part of the brain stripped from T1 weighted image. It is similar to overlaying masks to actual images.
- The resolution is 1 mm3 and each file is NiFTI (.nii.gz) format.

## Outline

- Prepare NFBS Data
- Data Generator to Feed Data to Model
- 3D UNet Model Architecture
- Train 3D UNet Model
- Deploy 3D UNet for Skull Stripping